In [1]:
import os
import datetime
import time
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.loader.dataloader import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch_geometric.data import Data
import torch_geometric
import networkx as nx

from sklearn.model_selection import train_test_split

from ClusterDataset import ClusterDataset as GNNDataset
from ClusterDatasetTransformer import ClusterDataset
from train_transformer import *
from data_statistics import *

from IPython.display import display

from Transformer import Transformer
from lang import Lang
from LossFunction import Loss

In [2]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


In [3]:
input_length = 60
max_seq_length = 60
batch_size = 64
max_nodes = 66

In [4]:
converter = Lang(max_nodes)
vocab_size = converter.n_words

In [ ]:
# Load the dataset
hist_folder = "/eos/user/c/czeh/histo_10pion0PU/"
data_folder_training = "/eos/user/c/czeh/graph_data"
data_folder_test = "/eos/user/c/czeh/graph_data_test"

dataset_training = GNNDataset(data_folder_training, hist_folder)
dataset_test = GNNDataset(data_folder_test, hist_folder, test=True)

data_folder_training = "/eos/user/c/czeh/graph_data/processed"
store_folder_training = "/eos/user/c/czeh/graph_data_trans"
data_folder_test = "/eos/user/c/czeh/graph_data_test/processed"
store_folder_test = "/eos/user/c/czeh/graph_data_trans_test"

model_folder = "/eos/user/c/czeh/model"
dataset_training = ClusterDataset(converter, store_folder_training, data_folder_training, max_nodes=max_nodes, input_length=input_length)
dataset_test = ClusterDataset(converter, store_folder_test, data_folder_test, max_nodes=max_nodes, input_length=input_length)

{run_: 1, luminosityBlock_: 1, event_: 1, NTracksters: 251, NClusters: ..., ...}
-2.3349583
-2.4997268
-2.5590408
-2.516017
-2.5506535
-2.3471353
-2.3592002
-2.3019857
-2.291667
-2.304002
-2.4282427
-2.2724545
-2.3856122
-2.387721
-2.3848696
-2.3392825
-2.4256113
-1.728149
-2.320049
-2.5238209
-2.3229148
-2.4321885
-2.3248706
-2.2913327
-2.2074893
-2.3986914
-2.3266127
-1.7659259
-1.7317032
-2.2631202
-2.3606215
-2.2945032
-2.314862
-2.2772572
-2.335772
-1.7750444
-2.3166306
-2.4061596
-2.272888
-1.8645442
-1.7647746
-2.4312994
-2.2971787
-1.7565992
-1.7130709
-2.0679872
-2.3908663
-2.0288582
-2.0964916
-1.8633878
-1.818656
-1.7524644
-2.3518035
-2.4250414
-2.4330144
-2.4753156
-2.2604473
-2.233069
-1.9761841
-1.9785763
-1.7388797
-2.3357422
-2.493187
-2.1886294
-2.3034868
-2.0504942
-2.0864372
-2.0118434
-2.0367951
-1.8424298
-1.7665558
-1.8028868
-1.7766178
-1.9999682
-2.0036895
-2.0331047
-2.4791439
-2.3292024
-2.4142175
-2.3494284
-1.9629556
-2.167693
-2.291444
-2.2453034
-2.260026

-1.8741837
-2.3170834
-2.4430635
-1.998882
-2.3869271
-1.8517787
-2.3382368
-1.880087
-1.7294527
-1.6428577
-2.6052654
-1.8219665
-2.3340065
-2.348205
2.8788319
2.6889412
2.2568612
2.3190725
2.6648216
2.2025368
2.518598
2.365929
2.288484
2.4145527
2.297254
2.7284484
2.452043
2.4325264
2.3712387
2.4151926
2.6999447
2.3737357
2.3416805
2.6866663
2.3566422
2.6012952
2.425706
2.527709
2.6819541
2.7039506
2.5384974
2.6820178
2.359225
2.5881197
2.1651156
2.3703516
2.3375652
2.7069957
2.2960093
2.4823284
2.0664806
2.5616236
2.6653726
2.1562119
2.1434162
3.0851364
2.5698092
2.1365402
2.4700663
2.1609266
2.5208378
2.183554
2.4539843
2.5294306
2.7894242
2.3878586
2.3036332
2.52709
2.3629215
2.4778485
2.422802
2.4507234
2.3488638
2.3350644
2.0868683
2.1334217
1.9174817
2.7215033
2.7755497
2.5953531
2.6246188
2.3643625
2.2465057
2.2479959
2.54807
2.3550327
2.4027646
2.3245614
2.17177
1.9264494
1.8751153
2.7904642
2.6166737
2.395645
2.3032455
2.3820646
2.4626966
2.3291912
2.6158035
2.1950288
2.2465

-2.4205754
-2.324368
-2.4529593
-2.5167384
-2.4241514
-2.173935
-2.1374836
-2.16156
-2.136291
-2.1001632
-2.0666583
-1.8482251
-1.7942683
-1.697862
-1.7590057
-1.7900988
-1.9941112
-1.9620525
-1.9838729
-2.8627357
-2.5282955
-2.6059792
-1.8294777
-2.047112
-1.7525235
-1.7202657
-2.0405872
-2.0614529
-2.916455
-2.792048
-2.2809181
-2.5476298
-2.5688887
-2.6323707
-2.3968396
-2.459907
-2.5165172
-2.1877317
-2.1991339
-1.7246488
-1.9637419
-2.0223887
-2.7279139
-2.7945435
-2.593652
-2.7380018
-2.6471128
-2.2025733
-2.1334736
-2.1315434
-1.813433
-2.0566022
-2.6470048
-2.62256
-2.6546123
-2.5382676
-2.5484023
-2.3338513
-2.5013802
-2.1553845
-2.0921328
-2.0701644
-1.753412
-2.5859041
-2.3252125
-2.2705429
-2.238321
-1.8718883
-2.0879583
-2.3690243
-2.6905317
-2.5737064
-2.3829703
-2.4822154
-2.284442
-2.5866194
-2.1550071
-2.208438
-2.3239217
-2.238886
-1.8538278
-2.676531
-2.6325405
-2.2739236
-2.7272167
-2.7689223
-2.6362212
-2.1296418
-2.1575873
-2.0282753
-1.7903275
-2.5525234
-2.65135

-1.9000114
-2.0533867
-2.5629878
-2.423911
-2.453358
-2.1968145
-2.0418236
-2.5395532
2.7646816
2.5568972
2.1661417
2.1766193
2.7710059
1.8754296
2.6423633
1.9748267
2.7563396
2.6728308
2.4730535
2.1089664
2.0019574
2.1217782
1.9306837
2.67426
2.7198112
1.9989369
2.6831226
2.6028993
2.1986294
2.2417727
2.63521
1.9560294
1.9857509
1.9514446
2.0563824
2.1650689
2.278769
2.3243244
2.6435876
2.2372136
2.15829
2.1671114
2.054946
2.7060704
2.2308424
2.216143
2.6130304
2.7051573
2.2330568
2.2494733
2.520218
2.255959
2.2040534
2.1866543
2.220087
2.2284925
2.1796749
2.1990972
2.1455765
2.0788424
2.150311
2.0192125
2.0089283
2.0266573
2.2316597
2.2148018
2.631594
2.398745
2.2692268
2.2420566
2.207346
2.1810286
2.1139953
2.1542242
2.1980882
2.1337237
2.1310291
2.039126
2.2873273
2.34339
2.2443404
2.4766788
2.5927565
2.63713
2.2881176
2.2274954
2.3832543
2.3532605
2.1747653
2.1697924
2.1010017
2.1271064
2.1909785
2.1055984
2.315401
2.7282333
2.9526093
2.3277175
2.5322948
2.5461912
2.4334002
2.2503

-2.1417823
-2.043011
-1.9990396
-2.3350437
-2.2855134
-2.0057466
-2.2257202
-2.077028
-2.2546144
-2.062805
-1.9671767
-2.040494
-2.2867754
-2.0067358
-2.1143703
-2.1145744
-1.9986528
-2.43903
-2.3507555
-2.0275288
-2.0540466
-2.3992069
-1.9805257
-2.0730104
-1.6604717
-2.1281817
-2.1261961
-2.0697858
-2.3251019
-2.1039386
-2.2166352
-1.7608515
-2.0375826
-2.1242383
-2.1429732
-2.0760796
-1.9465717
-2.043512
-2.017628
-1.907775
-1.7455301
-1.9487052
-2.1675608
-1.8418909
-1.9806795
-2.0258555
2.2931995
2.4637675
2.2216628
2.2855685
2.3368144
2.2462375
1.9749204
2.3135543
2.2873352
2.2793126
2.3507962
1.9524885
2.0214539
2.2503884
1.7659332
1.9181746
2.208943
2.0206854
2.001407
2.0460856
2.4054427
2.027282
2.0000517
1.9284728
2.6254425
2.4133723
2.002845
1.8552889
2.7121468
2.561782
2.3063052
2.0416486
2.1263402
2.1094098
2.322537
2.6782634
2.3907826
2.3695385
2.226289
2.6437511
2.539844
2.3530853
2.0883377
2.0347028
1.7941009
2.0284808
2.0050108
2.1132278
1.9493469
2.6388292
2.342002
2.

...

-2.011728
-1.9973024
-1.8695495
-2.0822394
-2.010649
-1.9683282
-2.0456433
-2.3915577
-2.0417626
-2.0921392
-2.0653036
-2.0808992
-2.4304383
-2.515707
-2.0298667
-2.6090846
-2.1106648
-2.107421
-2.4710984
-1.992168
-2.5736034
-2.0713985
-2.0635254
-2.0827036
-2.0325058
-2.03319
-2.0645862
-2.216231
-2.0169976
-2.0460222
-2.512929
-2.0192473
-2.124886
-2.0636337
-2.1072638
-2.5478666
-2.0979035
-2.1335192
-1.8597636
-2.0662508
-2.201481
-2.4569044
-2.3588023
-2.4240267
-2.3472025
-2.5954823
-2.0801349
-2.1510625
-1.9910452
-2.4176908
-2.5028906
-2.530493
-2.5870233
-2.4022038
-2.0518827
-2.0938127
-2.0307167
-2.097838
-2.1415453
-2.205701
-2.1471694
-2.054904
-2.02642
-2.0469995
-2.3577547
-2.5621223
-2.5426297
-2.4155557
-2.1272671
-2.1528296
-1.9279295
-1.8987244
-1.935086
-2.0196717
-2.2359164
-2.1141121
-2.0825484
-1.9622824
-1.9787028
-2.502364
-2.5253706
-2.4423506
-2.319528
-2.0963597
-1.9440675
-1.9832684
-2.4219153
-2.4780762
-2.5592625
-2.338189
-2.1271453
-2.1137688
-2.410997

2.2345703
1.7960814
1.9145927
2.3948317
2.5907772
2.4753196
1.941142
1.9498523
2.2841055
2.3393986
2.6266458
2.3020933
2.2327254
2.2330155
2.1596167
1.8135097
1.8634753
1.9674928
2.3769298
2.711362
2.558561
2.3469677
1.8094416
1.7824152
2.2722676
1.8907967
1.9875864
1.9677385
1.8555238
2.2497659
2.3128228
2.2631235
1.7589419
1.9001836
2.6130242
2.2699072
2.2942774
1.8928621
1.862445
2.6962228
2.3349106
2.321369
2.2340174
1.9316208
2.7627015
2.1946907
1.9230541
1.8515147
1.6960135
2.3340583
1.9103948
2.4421477
2.6063693
2.6659963
1.8427272
2.382633
2.3050625
1.9564637
2.5063703
2.379617
2.4433327
2.5713556
2.6739564
2.72074
2.3591104
2.320891
2.4500942
1.942231
1.7559291
2.2961383
1.9234207
1.8298098
2.3812435
2.636661
1.9454135
1.8898187
2.3574078
2.383456
2.2523527
1.9539095
{run_: 1, luminosityBlock_: 1, event_: 27, NTracksters: 191, ...}
-2.2534058
-2.592784
-2.563189
-2.6830082
-2.307106
-2.2295663
-2.2792034
-2.279346
-2.5716145
-2.3380764
-1.7102317
-2.2632506
-2.2808845
-2.28418

-2.2523298
-2.392435
-2.1131752
-2.1999326
-2.1691039
-2.1262486
-2.057299
-2.2287323
-2.3096013
-2.171958
-2.2035906
-2.0864637
-1.9519336
-1.9259144
-2.318743
-2.3628511
-2.4138896
-2.227906
-2.1838639
-2.2691977
-2.1602476
-2.167715
-2.084592
-1.9346939
-1.9837172
-1.9449356
-2.1030955
-2.375722
-2.3536336
-2.3877041
-2.1963325
-2.0140803
-1.9588296
-1.8558507
-2.1254454
-2.1452198
-1.9165972
-2.155771
-2.1859486
-2.139637
-2.1367106
-2.1697302
-2.1034126
-2.1522462
-2.1221807
-2.288166
-2.1118839
-2.1467671
-2.1003892
-2.070078
-2.119111
-2.2273712
-2.1950252
-2.088385
-1.8989134
-2.3450556
-2.1729317
-2.1848714
-2.1116416
-1.8581103
-2.3387191
-2.1832092
-2.1709387
-1.859006
-2.1652405
-2.1362586
-2.1347234
1.9884708
1.6896169
2.113289
1.9088539
1.7613311
2.1826293
1.7184814
2.1371655
1.7523097
1.9765197
2.2938135
2.1035347
1.9573972
2.1886485
1.98646
2.1718402
2.131295
1.8997986
1.929906
2.0668383
2.0975358
1.7411156
2.1364725
2.285435
2.100193
1.7736946
2.2729821
2.1323047
2.167

1.7738235
2.5401533
2.3888347
2.4961383
2.4495614
2.4345016
2.4049995
1.848698
1.7461071
1.7697791
1.9850061
1.982344
2.0145729
2.0897415
2.3782873
2.4107406
2.374661
1.7997338
1.829551
1.9812243
1.7938727
1.9948102
1.8415055
2.4834666
2.5281913
2.4277189
2.442318
2.3979573
1.845736
1.8259449
1.7633473
1.7635193
2.0148256
1.6408534
1.8278699
1.7758902
1.7239504
2.4368024
1.8671776
2.262546
1.7384917
2.5037882
1.7991843
2.4144316
2.4391494
1.852493
{run_: 1, luminosityBlock_: 1, event_: 36, NTracksters: 260, ...}
-2.2643907
-2.2538733
-2.016883
-2.3356733
-2.2623641
-2.2808976
-2.2298207
-2.1768188
-1.9409133
-2.0754244
-1.9004759
-2.3061726
-2.5805907
-2.2838082
-2.2172542
-2.3009422
-2.3252575
-2.4302588
-2.2542715
-2.3898687
-2.4161115
-1.8949585
-2.510517
-2.256013
-2.444086
-1.8381788
-2.5161135
-2.566196
-2.3364758
-2.5751615
-1.9981625
-1.794976
-2.3063264
-1.8376994
-2.4665065
-2.467285
-2.3465276
-2.4452412
-2.3076131
-2.2779453
-1.8935301
-2.4146502
-2.4219162
-2.3894901
-2.28

-2.5710819
-2.5993676
-2.780364
-2.1038601
-2.1470072
-2.2464666
-2.2341652
-2.6589928
-2.177658
-2.2232244
-2.09964
-2.6956174
-2.7561655
-2.5966353
-2.6091478
-2.5748322
-2.6411505
-2.5489883
-2.5665529
-2.7000525
-2.6400986
-2.6198812
-2.6073155
-2.0387983
-1.9184449
-1.8920921
-2.618588
-2.480827
-2.477272
-2.6180494
-2.2046633
-2.2408507
-2.4908838
-2.5608907
-1.7663726
-2.1327703
-2.6041129
-2.6603792
-1.7534872
-2.6027315
-2.5286076
-2.118599
-2.6533592
-2.668821
-2.1524904
-2.6837947
-2.6949375
-2.7105763
-2.6377985
-1.7554654
-2.6360352
-2.6543868
-2.5812037
-2.096131
-2.6373527
-2.7446206
-2.7012942
-2.1302834
-2.0563695
-2.434539
-2.1462379
-2.088436
2.8443227
2.4878814
2.757844
2.4665594
2.777728
2.6615763
2.4648669
2.6147723
2.1437948
2.6849136
2.0758436
2.1400561
1.7837422
2.672552
2.593019
2.6383455
2.0952554
2.4991398
2.6325834
2.6221516
2.9039364
2.59577
2.4611628
2.6055765
2.056205
2.1872764
2.0589852
2.4396906
2.0997295
2.6653955
2.3164384
2.6385343
2.500097
2.911514

In [ ]:
train_dl = DataLoader(dataset_training, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=4)
test_dl = DataLoader(dataset_test, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=4)

In [ ]:
epochs = 100
d_model = 128
num_heads = 4
num_layers = 6
d_ff = 256
dropout = 0.2
padding = converter.word2index["<PAD>"]
feature_num = len(dataset_test.model_feature_keys)

# Model, loss, and optimizer
model = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, feature_num, max_nodes, max_seq_length, dropout).to(device)
criterion = Loss(converter)

In [ ]:
# Optionally introduce weight decay
# optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

# Drop Step Size over time
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
#Load Weights if needed
# weights = torch.load("/eos/user/c/czeh/tranformer_2.pt", weights_only=True)
# model.load_state_dict(weights["model_state_dict"])
# optimizer.load_state_dict(weights["optimizer_state_dict"])
# start_epoch = weights["epoch"]

In [ ]:
train_loss_hist = []
val_loss_hist = []

In [ ]:
# https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
# Optionally introduce gradient clipping
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.25)

fig_loss, ax_loss = plt.subplots(1, 1)
fig_loss.set_figwidth(6)
fig_loss.set_figheight(3)

display_loss = display(1, display_id=True)

optimizer.zero_grad()
# Training loop
for epoch in range(1, 101):
    print(f'Epoch: {epoch}')
    
    loss = train(model, converter, optimizer, train_dl, epoch, loss_obj=criterion, device=device)
    print(f"Training loss: {loss}")
    train_loss_hist.append(loss)
    
    val_loss = test(model, converter, test_dl, epoch, loss_obj=criterion, device=device)
    val_loss_hist.append(val_loss)
    print(f"Validation loss: {val_loss}")
    
    ax_loss.clear()
    plot_loss(train_loss_hist, val_loss_hist, ax=ax_loss, n=1)
    display_loss.update(fig_loss)
    time.sleep(1)
    
    scheduler.step()
    print(f"Epoch {epoch+1}, LR: {scheduler.get_last_lr()[0]}")

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_figheight(6)
fig.set_figwidth(10)
epochs = len(train_loss_hist)
ax.plot(range(1, epochs+1), moving_average(train_loss_hist, 8), label='train', linewidth=2)
ax.plot(range(1, epochs+1), moving_average(val_loss_hist, 8), label='val', linewidth=2)
ax.set_ylabel("Loss", fontsize=14)
ax.set_xlabel("Epochs", fontsize=14)
ax.set_title("Training and Validation Loss", fontsize=14)
ax.legend()

In [ ]:
date = f"{datetime.now():%Y-%m-%d}"
save_model(model, epoch, optimizer, train_loss_hist, val_loss_hist, model_folder, f"tranformer_date_{date}.pt")

## Test Full Event

In [ ]:
from EventGrouping import EventGrouping

In [ ]:
model2 = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, feature_num, max_nodes, max_seq_length, dropout).to(device)
weights = torch.load("/eos/user/c/czeh/tranformer_4.pt", weights_only=True)
model2.load_state_dict(weights["model_state_dict"])

In [ ]:
runner = EventGrouping(converter, model2, neighborhood=1, seq_length=input_length)
runner(dataset_test.get(0))

In [ ]:
model_feature_keys = np.array([0,  2,  3,  4,  6,  7, 10, 14, 15, 16, 17, 18, 22, 24, 25, 26, 28, 29])
dataset_training.__getitem__(0)[0][:, model_feature_keys][:, -1]

In [ ]:
dataset_test.get(0).cluster

In [ ]:
dataset_training.node_feature_keys[16]

In [ ]:
G = torch_geometric.utils.to_networkx(dataset_test.get(0), to_undirected=True)

In [ ]:
dataset_training.get(0).x[:, 16]

In [ ]:
fig, ax = plt.subplots(1, 1)
nx.draw(G, with_labels=True, ax=ax)

## Random Tests

In [ ]:
targets = dataset_training.__getitem__(0)[2]
mask = targets != -4
mask.unsqueeze(-1)

In [ ]:
targets[targets[:, -1] != -4, :]

In [ ]:
targets[mask].shape[0]/3

In [ ]:
opts = dataset_training.__getitem__(0)[1]
opts = torch.roll(opts, -1, dims=0)
opts[-1] = 5
opts

In [ ]:
out_mask = opts != -4
opts[out_mask].shape[0]

In [ ]:
targets = torch.reshape(targets[mask], (int(targets[mask].shape[0]/3), 3))

In [ ]:
targets[0, :]